In [1]:
from whisperweranalysis.structure import LJSpeech
from whisperweranalysis.groundtruth import lj_valid
from whisperweranalysis.wer import compute

/home/shivam/Projects/analysis-scripts/WhisperWERAnalysis/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from whisperweranalysis.mel_utilities import audio2mel

In [3]:
# audio2mel()

In [4]:
%load_ext autoreload
%autoreload 2

## Natural Speech

In [14]:
GT = LJSpeech.load_from_dir('../whisperweranalysis/LJ_Valid_transcription/GT/0')

In [15]:
compute(lj_valid, GT.transcriptions)

2.4198427102238353

## Vocoded Speech

In [16]:
GT_Mel = LJSpeech.load_from_dir('../whisperweranalysis/LJ_Valid_transcription/GT/GT_MEL')

In [17]:
compute(lj_valid, GT_Mel.transcriptions)

1.9963702359346642

In [18]:
with open('temp_NS.txt', 'w') as f:
    f.writelines([f'{x}\n' for x in GT.transcriptions])
    
with open('temp_NS_Vocoded.txt', 'w') as f:
    f.writelines([f'{x}\n' for x in GT_Mel.transcriptions])
    